In [1]:
# Función que etiqueta el conteo final encima de cada barra del gráfico
def label_values(ax, spacing=5):
    total = 0
    for rect in ax.patches:
        total += rect.get_height()

    for rect in ax.patches:
        y_value = rect.get_height()
        x_value = rect.get_x() + rect.get_width() / 2

        space = spacing
        
        va = 'bottom'
        
        if y_value < 0:
            space *= -1
            va = 'top'
        label = "{:.2f},  {:.2f} %".format(y_value, y_value/total*100)
        ax.annotate(
            label,                      
            (x_value, y_value),         
            xytext=(0, space),          
            textcoords="offset points", 
            ha='center',                
            va=va)     

In [2]:
# Función que detecta los outliers de una feature
def outliers_indices(feature):
    # 1. Calcula la media de la feature
    mid = df[feature].mean()
    # 2. Calcula la desviacion estandard de la feature
    sigma = df[feature].std()
    # 3. Se imprime la media y la estandard por pantalla
    print(f'Feature {feature} , media: {mid}, sigma: {sigma}')
    # 4. Retorna aquellos ínices que están muy alejados de 3 sigma con respecto a la media
    return df[(df[feature] < mid - 3*sigma) | (df[feature] > mid + 3*sigma)].index

In [3]:
# Función que procesa el data_frame original

def preprocess(df): 
    
    df = df.rename(columns={'y': 'deposit','cons.price.idx':'ipc','default':'debts','housing':'mortgage',
                       'duration':'call_duration','campaign':'n_contacts','previous':'n_past_contacts',
                        'poutcome':'previous_results', 'pdays': 'days_from_last_campaign','cons.conf.idx':'icc',
                        'nr.employed':'n_employed','emp.var.rate':'emp_var_rate'})
       
    # 1. Se filtran las numéricas para su estandarización posterior
    df_numerical = df.select_dtypes(exclude="object") 
    
    # 2. Se binariza la variable target deposit
    binarize_n = {'yes':  1, 'no': 0} 
    df['deposit_n'] = df['deposit'].map(binarize_n)
    
    # 3. Se codifican las variables debts, mortgage y loan mediante el método LabelEncoder
    #coding_n = {'unknown': 0, 'yes': 1, 'no': 2}
    from sklearn. preprocessing import LabelEncoder
    le = LabelEncoder()
    le.fit(df.debts)
    le.fit(df.mortgage)
    le.fit(df.loan)
    df['debts_n'] = le.transform(df.debts)
    df['mortage_n'] = le.transform(df.mortgage)
    df['loan_n'] = le.transform(df.loan)

    # 4. Se codifican los meses del 1 al 12 secuencialmente 
    coding_mon = {'jan': 1, 'feb': 2, 'mar': 3, 'apr': 4, 'may': 5, 'jun': 6, 'jul': 7, 'aug': 8, 'sep': 9, 'oct': 10, 'nov': 11, 'dec':12}
    df['month_n'] = df['month'].map(coding_mon)

    # 5. Se codifica las variables contact y previous_results según la siguiente codificación
    coding_contactd_cont = {'unknown': 0, 'cellular': 1, 'telephone': 2}
    df['contact_n'] = df['contact'].map(coding_contactd_cont)
    coding_previous_results = {'nonexistent':0, 'failure':1, 'success':2}
    df['previous_results_n'] = df['previous_results'].map(coding_previous_results)

    # 6. Se codifica el resto de variables como one hot encoding : job, marital, education y day_of_week, 
    df = pd.get_dummies(df, columns = ['job', 'marital', 'education', 'day_of_week'],drop_first=True)
    
    # 7. Se eliman algunas variables ya codificadas
    df = df.drop(['deposit', 'debts','mortgage', 'loan','month', 'contact', 'previous_results'], axis = 1)
    
    # 8. Se estandariza todas las variables numéricas
    
    from sklearn.preprocessing import StandardScaler
    scaler = StandardScaler()
    #scaler = MinMaxScaler()
    
    df[df_numerical] = scaler.fit_transform(df[df_numerical])
    
    # 9. Se retorna el data_frame procesado.
    return df

In [4]:
# Función que procesa el data_frame original como entrada 
# al modelo ya entrenado. 

def transform (df,rename=False,scaler=True):
    # 1. Se renombran las columnas originales 
    if rename:
        df = df.rename(columns={'y': 'deposit','cons.price.idx':'ipc','default':'debts','housing':'mortgage',
                       'duration':'call_duration','campaign':'n_contacts','previous':'n_past_contacts',
                        'poutcome':'previous_results', 'pdays': 'days_from_last_campaign','cons.conf.idx':'icc',
                        'nr.employed':'n_employed','emp.var.rate':'emp_var_rate'})
    
    # 2. Se filtran las numéricas para su estandarización posterior
    df_numerical = df.select_dtypes(exclude="object") 
    
    # 3. Se eliminan aquellas variables numéricas que no aportan valor predictivo. 
    df.drop(['emp_var_rate', 'n_employed','call_duration'], axis = 1, inplace=True)
    df_numerical = list(set(df_numerical) - set(['emp_var_rate', 'n_employed','call_duration']))
    
    # 4. Se binariza la variable target deposit
    binarize_n = {'yes':  1, 'no': 0} 
    df['deposit_n'] = df['deposit'].map(binarize_n)
    
    # 5. Se codifican las variables debts, mortgage y loan mediante el método LabelEncoder
    #coding_n = {'unknown': 0, 'yes': 1, 'no': 2}
    from sklearn. preprocessing import LabelEncoder
    le = LabelEncoder()
    le.fit(df.debts)
    le.fit(df.mortgage)
    le.fit(df.loan)
    df['debts_n'] = le.transform(df.debts)
    df['mortage_n'] = le.transform(df.mortgage)
    df['loan_n'] = le.transform(df.loan)

    # 6. Se codifican los meses del 1 al 12 secuencialmente 
    coding_mon = {'jan': 1, 'feb': 2, 'mar': 3, 'apr': 4, 'may': 5, 'jun': 6, 'jul': 7, 'aug': 8, 'sep': 9, 'oct': 10, 'nov': 11, 'dec':12}
    df['month_n'] = df['month'].map(coding_mon)

    # 7. Se codifica las variables contact y previous_results según la siguiente codificación
    coding_contactd_cont = {'unknown': 0, 'cellular': 1, 'telephone': 2}
    df['contact_n'] = df['contact'].map(coding_contactd_cont)
    coding_previous_results = {'nonexistent':0, 'failure':1, 'success':2}
    df['previous_results_n'] = df['previous_results'].map(coding_previous_results)

    # 8. Se codifica el resto de variables como one hot encoding : job, marital, education y day_of_week, 
    df = pd.get_dummies(df, columns = ['job', 'marital', 'education', 'day_of_week'],drop_first=True)
    
    # 9. Se eliman algunas variables ya codificadas
    df = df.drop(['deposit', 'debts','mortgage', 'loan','month', 'contact', 'previous_results'], axis = 1)
    
    # 10. Se estandariza todas las variables numéricas
    if scaler:
        from sklearn.preprocessing import StandardScaler
        scaler = StandardScaler()
        #scaler = MinMaxScaler()
    
        df[df_numerical] = scaler.fit_transform(df[df_numerical])
    
    # 11. Se retorna el data_frame procesado.
    return df

In [5]:
# Realiza un infome de clasificación del modelo 
def report(model):
    # 1. Realiza una predicción del modelo
    preds = model.predict(x_test)
    # 2. Imprime el informe de clasificación en base a las predicciones
    print(classification_report(preds,y_test))
    #print("Precision score: {}".format(precision_score(preds,y_test)))
    # 3. Obtiene la matriz de consfusión 
    cm = confusion_matrix(y_test, preds, labels=model.classes_)
    
    #plot_confusion_matrix(model,x_test,y_test)
    # 4. Plotea la matriz de confusión
    disp=  ConfusionMatrixDisplay(confusion_matrix=cm,
                              display_labels=model.classes_)
    disp.plot()
    #plot_precision_recall_curve(model,x_test,y_test)
    #plot_roc_curve(model,x_test,y_test)



In [6]:
# Entrenamiento del modelo usando el tuneado de hiper-parámetros
def training_model_hyperparameter(model, scoring, params_grid, X_train, y_train):
    # 1. Se define la validación cruzada repetidas con 10 splits y en modo aleatorio
    folds = StratifiedKFold(n_splits = 10, shuffle = True, random_state=100)

    # 2. Define la malla especificando hiper-parámetros, estimador, validación cruzada y métrica asociada.
    grid = GridSearchCV(estimator = model, scoring=scoring, param_grid = params_grid, cv = folds, 
                           verbose=0, return_train_score=True, n_jobs=3)
    # 3. Entrenamiento del modelo con la configuración anterior. 
    grid.fit(X_train, y_train)
    # 4. Se retorna el modelo entrenado.
    return grid

In [7]:
# Calculo predicción y probabilidades en datos de entrenamiento y test.
def prediction_model(model, X_train, y_train, X_test, y_test):
    # 1. Cálculo predicción de datos de entrenamiento
    y_train_pred = model.predict(X_train)
    # 2. Cálculo probabilidades de datos de entrenamiento
    y_train_pred_prob = model.predict_proba(X_train)[:, 1]
    # 3. Cálculo predicción de datos de
    y_test_pred = model.predict(X_test)
    # 4. Cálculo probabilidades de datos de test
    y_test_pred_prob = model.predict_proba(X_test)[:, 1]
    # 5. Retorna los cálculos realizados. 
    return y_train_pred, y_train_pred_prob, y_test_pred, y_test_pred_prob



In [8]:
# Curva ROC de la probabilidad de los datos de test y entrenamiento 
def draw_roc( train_actual, train_probs, test_actual, test_probs ):
    train_fpr, train_tpr, train_thresholds = metrics.roc_curve( train_actual, train_probs,
                                              drop_intermediate = False )
    test_fpr, test_tpr, test_thresholds = metrics.roc_curve( test_actual, test_probs,
                                              drop_intermediate = False )
    train_auc_score = metrics.roc_auc_score( train_actual, train_probs )
    test_auc_score = metrics.roc_auc_score( test_actual, test_probs )
    plt.figure(figsize=(5, 5))
    plt.plot( train_fpr, train_tpr, label='ROC curve (area = %0.2f)' % train_auc_score )
    plt.plot( test_fpr, test_tpr, label='ROC curve (area = %0.2f)' % test_auc_score )
    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate or [1 - True Negative Rate]')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver operating characteristic example')
    plt.legend(loc="lower right")
    plt.show()

In [9]:
# Importancia de vbles predictoras con RF

def rf_feat_importance(m, df):
    return pd.DataFrame({'cols':df.columns, 'imp':m.feature_importances_}
                       ).sort_values('imp', ascending=False)

# Ploteado de la importancia de vbles predictoras con RF 

def plot_fi(fi):
    return fi.plot('cols', 'imp', 'barh', figsize=(12,7), legend=False)




In [11]:
# Realiza un infome de clasificación del modelo para un threshold de 0.3
def report_threshold(model):
    # 1. Cálculo de las predicciones para un umbral de 0.3
    preds = (model.predict_proba(x_test)[:,1] >=0.3).astype(bool)
    # 2. Se imprime el inforrme de clasificación
    print(classification_report(preds,y_test))
    # 3. Se calcula la matriz de consusión
    cm = confusion_matrix(y_test, preds, labels=model.classes_)
    
    #4. Se plotea la matriz de confusión.
    disp=  ConfusionMatrixDisplay(confusion_matrix=cm,
                              display_labels=model.classes_)
    disp.plot()